In [79]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

In [81]:
def get_game(url):
    random_number = random.randint(0, 7)
    time.sleep(random_number)
    bets = []
    service = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    game_title = soup.find('span', class_='o_match-title').text.strip()
    time.sleep(7 + random_number)  
   
    accordion_button = driver.find_element(By.XPATH, "//button[@class='o_accordion-item__trigger']//span[contains(text(), 'Genaues Ergebnis')]")
    accordion_button.click()

    time.sleep(5)  

    accordion_item = accordion_button.find_element(By.XPATH, "./ancestor::div[contains(@class, 'o_accordion-item')]")

    if 'o_accordion-item--expanded' in accordion_item.get_attribute('class'):
        panel = accordion_item.find_element(By.CLASS_NAME, 'o_accordion-item__panel')
        panel_html_content = panel.get_attribute('innerHTML')
        panel_soup = BeautifulSoup(panel_html_content, 'html.parser')
        bet_items_section = panel_soup.find_all('div', class_='o_bet-item')

        for section in bet_items_section:
            header = section.find('div', class_='o_bet-item__header').text.strip()
            if "Genaues Resultat (reg.Spielzeit)" in header:
                bet_items = section.find('div', class_='o_bet-item__content ng-star-inserted').find_all('button', class_='m_bet-button')
                for item in bet_items:
                    name = item.find('span', class_='m_bet-button__name').text.strip()
                    odds = item.find('span', class_='m_bet-button__odds-text').text.strip()
                    bets.append((name, odds))
                break
    else:
        print("Accordion item is not expanded.")


    driver.quit()
    return bets,game_title
    

In [82]:
all_games = []
with open('list_urls.txt', 'r') as file:
    all_games = [line.strip() for line in file.readlines()]

In [83]:
def call_five_times(url): 
    bets = []
    game_title = ""
    for i in range(5):
        try: 
            bets, game_title = get_game(url)
            if bets:
                return bets, game_title
        except Exception as e:
            print(f"Attempt {i+1} failed for game {url}")

In [84]:
all_bets_data = []
failed = []
with ThreadPoolExecutor(max_workers=7) as executor:
    future_to_game = {executor.submit(call_five_times, url): url for url in all_games}

    for i, future in enumerate(as_completed(future_to_game)):
        game_url = future_to_game[future]
        print(f"Crawling game {i+1}")
        try:
            bets, game_title = future.result()
            if bets:
                for name, odds in bets:
                    all_bets_data.append([game_title, name, float(odds)])
            else:
                failed.append(game_url)
        except Exception as e:
            print(f"Error occurred for game {game_url}: {e}")
            failed.append(game_url)



Attempt 1 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2634/event/519421
Attempt 2 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2634/event/519421
Attempt 1 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2632/event/497258
Attempt 3 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2634/event/519421
Attempt 1 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2632/event/497242
Attempt 4 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2634/event/519421
Attempt 2 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/c

In [ ]:
df = pd.DataFrame(all_bets_data, columns=["Game Title", "Result", "Odds"])
# Sort the DataFrame by game title and odds
df_sorted = df.sort_values(by=["Game Title", "Odds"])

# Keep only the 3 lowest odds for each game title
df_filtered = df_sorted.groupby("Game Title").head(3)

# Display the filtered DataFrame
display(df_filtered)



,Game Title,Result,Odds
1,Deutschland – Schottland,2:0,5.11
0,Deutschland – Schottland,1:0,5.84
3,Deutschland – Schottland,3:0,6.54
